# Importaciones

In [ ]:
"""
Full demo script exercising all components of the Trading Environment repo.
"""
import logging
import optuna
from optuna.logging import set_verbosity, ERROR
import numpy as np

from trading_environment import (
    DataManager, RiskManager, PerformanceAnalyzer, Optimizer,
    ReportManager, TradingSystem, StrategyManager, StrategyCollection,
    DATA_DIR, MAX_FILE_SIZE
)
from strategies import (
    MovingAverageCrossStrategy, DcaStrategy, DonchianBreakoutStrategy,
    ADXTrendStrategy, ROCStrategy, MACDStrategy, BollingerMeanRevStrategy,
    RSIStrategy, PairTradingStrategy, VWAPStrategy
)

ImportError: attempted relative import with no known parent package

# Tests

In [ ]:
# Suppress less important logs
logging.getLogger().setLevel(logging.ERROR)  # hide WARNING and INFO logs
set_verbosity(ERROR)  # hide Optuna warnings

In [ ]:
# 1) Load data
dm = DataManager(DATA_DIR, MAX_FILE_SIZE)
data = dm.load_data(symbols='BTC-USD', period='2y', interval='1d')
print("Data loaded shape:", data.shape)

In [ ]:
# 2) RiskManager Demo
price = data['close'].iloc[-1]
rm_pct = RiskManager(method='pct', risk_pct=0.02)
print("Pct size:", rm_pct.calculate_size(10000, price, data))
rm_atr = RiskManager(method='atr', risk_pct=0.01, atr_window=14)
print("ATR size:", rm_atr.calculate_size(10000, price, data, stop_loss=price*0.95))

In [ ]:
# 3) Strategy Demo
strat = MovingAverageCrossStrategy({'short_window':10, 'long_window':20})
sig = strat.generate_signals(data)
print("Signal counts:", sig.value_counts().to_dict())
eq = strat.backtest(data, initial_capital=10000, transaction_cost=0.001)
pa = PerformanceAnalyzer(eq, strat.returns)
print("MA Crossover Performance:\n", pa.summary())

In [ ]:
# 4) Optimizer Demo (Grid, Bayes, DE)
print("\n=== Optimizer Demo ===")
subset = [MovingAverageCrossStrategy, MACDStrategy]
# Grid Search
print("-- Grid Search --")
best_strat_g, best_params_g, perf_g, eq_g = Optimizer.find_best_strategy(
    subset, data, method='grid', metric='Sharpe'
)
print(f"Grid best: {best_strat_g.name} {best_params_g}")
# Bayesian Search
print("-- Bayesian Search --")
best_strat_b, best_params_b, perf_b, eq_b = Optimizer.find_best_strategy(
    subset, data, method='bayes', metric='Sharpe', n_trials=10, seed=123
)
print(f"Bayes best: {best_strat_b.name} {best_params_b}")
# Differential Evolution
print("-- DE Search --")
best_strat_d, best_params_d, perf_d, eq_d = Optimizer.find_best_strategy(
    subset, data, method='de', metric='Sharpe', maxiter=5, popsize=5
)
print(f"DE best: {best_strat_d.name} {best_params_d}")

In [ ]:
# 5) ReportManager plotting & saving
rm = ReportManager()
print("Plotting metrics for Grid best...")
rm.plot_metrics(perf_g)

# --- FIX: extract the single Series for each best equity curve ---
eq_series_g = eq_g[best_strat_g.name]
eq_series_b = eq_b[best_strat_b.name]
eq_series_d = eq_d[best_strat_d.name]

print("Overlaying equity curves...")
rm.plot_equity_curves({
    'Grid (best)':  eq_series_g,
    'Bayes (best)': eq_series_b,
    'DE (best)':    eq_series_d
})

# Compare and save
returns_g = eq_series_g.pct_change().fillna(0)
returns_b = eq_series_b.pct_change().fillna(0)
returns_d = eq_series_d.pct_change().fillna(0)

analyzers = {
    'Grid':  PerformanceAnalyzer(eq_series_g, returns_g),
    'Bayes': PerformanceAnalyzer(eq_series_b, returns_b),
    'DE':    PerformanceAnalyzer(eq_series_d, returns_d)
}

comp = rm.compare_performance(
    analyzers,
    save=True,
    report_name='full_demo_comparison'
)
print(comp)

In [ ]:
# 6) TradingSystem Demo
print("\n=== TradingSystem Demo ===")
ts = TradingSystem('BTC-USD', DcaStrategy({'amount':500,'freq':'D'}), risk_manager=RiskManager('fixed', fixed_size=1))
ts.load(period='1mo', interval='1d')
eq_ts = ts.run_backtest(save_report=True, do_print=True, report_name='ts_demo')
print(f"Final TS equity: {eq_ts.iloc[-1]:.2f}")

In [ ]:
# 7) StrategyCollection Demo
grid_params = {'short_window': [5,10], 'long_window': [20,50]}
collection = StrategyCollection(strategy_cls=MovingAverageCrossStrategy, param_grid=grid_params)
eqs = collection.backtest_all('BTC-USD', period='1mo', interval='1d')
print("Collection demos backtested strategies:", list(eqs.keys()))

In [ ]:

# 8) StrategyManager Demo
sm = StrategyManager()
strat_sample = MACDStrategy({'fast':12,'slow':26,'signal':9})
sm.save(strat_sample)
loaded = sm.load(strat_sample.name)
print("Loaded sample strategy:", loaded.name)


# END